In [3]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import sqlite3
from datasets import load_dataset
from bs4 import BeautifulSoup
import re

In [5]:
# Load IMDB dataset from Hugging Face
dataset = load_dataset("imdb")
train_data = dataset["train"]
test_data = dataset["test"]

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:
# Combine train and test data using concatenate_datasets
from datasets import concatenate_datasets

all_reviews = concatenate_datasets([train_data, test_data])

In [8]:
# Initialize SQLite database
conn = sqlite3.connect('imdb_reviews.db')
cursor = conn.cursor()


In [9]:
# Create table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS imdb_reviews (
        id INTEGER PRIMARY KEY,
        review_text TEXT,
        sentiment TEXT
    )
''')

In [10]:
# Preprocess text (lowercase, remove HTML tags, punctuation)
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)                   # Remove punctuation
    return text.lower().strip()

In [13]:
# Insert data into the database
for idx, example in enumerate(all_reviews):
    cleaned_text = clean_text(example["text"])
    sentiment = "positive" if example["label"] == 1 else "negative"
    cursor.execute('''
        INSERT INTO imdb_reviews (id, review_text, sentiment)
        VALUES (?, ?, ?)
    ''', (idx, cleaned_text, sentiment))

<ipython-input-10-1eac734b2e48>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags


In [14]:
conn.commit()
conn.close()